<a href="https://colab.research.google.com/github/bonareri/Bitcoin-Prediction-Analysis/blob/main/streamlit_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#  Install Dependencies
!pip install streamlit plotly pandas prophet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.6 MB/s eta 0:00:00


### Load & Prepare Data

In [25]:
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [105]:
# Load Data
df = pd.read_csv("/content/predictions.csv")

In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ds          366 non-null    object 
 1   yhat        366 non-null    float64
 2   yhat_lower  366 non-null    float64
 3   yhat_upper  366 non-null    float64
dtypes: float64(3), object(1)
memory usage: 11.6+ KB


In [107]:
# Convert date column to datetime
df['ds'] = pd.to_datetime(df['ds'])

### Relative Strength Index (RSI)

Detects Overbought/Oversold Conditions

In [108]:
def calculate_rsi(df, window=14):
    delta = df['yhat'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df

df = calculate_rsi(df)

### Moving Averages (MA)

In [109]:
df['SMA_50'] = df['yhat'].rolling(window=50).mean()  # 50-day Simple Moving Average
df['SMA_100'] = df['yhat'].rolling(window=100).mean()  # 100-day Simple Moving Average

### Bollinger Bands
To measure price volatility:

In [110]:
df['Middle_Band'] = df['yhat'].rolling(window=20).mean()
df['Upper_Band'] = df['Middle_Band'] + 2 * df['yhat'].rolling(window=20).std()
df['Lower_Band'] = df['Middle_Band'] - 2 * df['yhat'].rolling(window=20).std()

### MACD (Moving Average Convergence Divergence)
a momentum or oscillator indicator that compares two distinct moving averages of a crypto asset to recognize

In [111]:
df['EMA_12'] = df['yhat'].ewm(span=12, adjust=False).mean()
df['EMA_26'] = df['yhat'].ewm(span=26, adjust=False).mean()
df['MACD'] = df['EMA_12'] - df['EMA_26']
df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()

### Sharpe Ratio
To measure risk-adjusted returns:

In [112]:
risk_free_rate = 0.01  # Assume a 1% risk-free rate
# Calculate additional risk metrics
df['Daily Return'] = df['yhat'].pct_change()
df['Excess_Return'] = df['Daily Return'] - risk_free_rate / 252
df['Sharpe_Ratio'] = df['Excess_Return'].mean() / df['Excess_Return'].std()

### Maximum Drawdown
To analyze the worst decline from peak to trough:

In [113]:
df['Cumulative_Max'] = df['yhat'].cummax()
df['Max_Drawdown'] = (df['yhat'] - df['Cumulative_Max']) / df['Cumulative_Max']

### Trend Analysis

In [114]:
df['Rolling_Volatility'] = df['Daily Return'].rolling(window=7).std()

In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ds                  366 non-null    datetime64[ns]
 1   yhat                366 non-null    float64       
 2   yhat_lower          366 non-null    float64       
 3   yhat_upper          366 non-null    float64       
 4   RSI                 353 non-null    float64       
 5   SMA_50              317 non-null    float64       
 6   SMA_100             267 non-null    float64       
 7   Middle_Band         347 non-null    float64       
 8   Upper_Band          347 non-null    float64       
 9   Lower_Band          347 non-null    float64       
 10  EMA_12              366 non-null    float64       
 11  EMA_26              366 non-null    float64       
 12  MACD                366 non-null    float64       
 13  Signal_Line         366 non-null    float64       

In [116]:
df_cleaned = df.dropna()

In [117]:
print(df_cleaned.isnull().sum())

ds                    0
yhat                  0
yhat_lower            0
yhat_upper            0
RSI                   0
SMA_50                0
SMA_100               0
Middle_Band           0
Upper_Band            0
Lower_Band            0
EMA_12                0
EMA_26                0
MACD                  0
Signal_Line           0
Daily Return          0
Excess_Return         0
Sharpe_Ratio          0
Cumulative_Max        0
Max_Drawdown          0
Rolling_Volatility    0
dtype: int64


In [118]:
#save cleaned data
df_cleaned.to_csv('cleaned_data.csv', index=False)

## Build the Streamlit Dashboard

In [126]:
%%writefile dashboard.py

# Load data
@st.cache_data
def load_data():
    df = pd.read_csv("/content/cleaned_data.csv", parse_dates=["ds"])
    return df

df = load_data()

# Page title
st.title("📊 Bitcoin Analysis Dashboard")

# Sidebar filters
st.sidebar.header("Filter Data")
date_range = st.sidebar.date_input("Select Date Range", [df["ds"].min(), df["ds"].max()])
filtered_df = df[(df["ds"] >= pd.to_datetime(date_range[0])) & (df["ds"] <= pd.to_datetime(date_range[1]))]

# Bitcoin Price Forecast
st.subheader("📈 Bitcoin Price Forecast")
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=filtered_df["ds"], y=filtered_df["yhat"], mode='lines', name='Predicted Price'))
fig1.add_trace(go.Scatter(x=filtered_df["ds"], y=filtered_df["yhat_lower"], mode='lines', name='Lower Bound', line=dict(dash='dot')))
fig1.add_trace(go.Scatter(x=filtered_df["ds"], y=filtered_df["yhat_upper"], mode='lines', name='Upper Bound', line=dict(dash='dot')))
st.plotly_chart(fig1)

# RSI Indicator
st.subheader("📊 Relative Strength Index (RSI)")
fig2 = px.line(filtered_df, x="ds", y="RSI", title="RSI Over Time", labels={"RSI": "Relative Strength Index"})
st.plotly_chart(fig2)

# Moving Averages
st.subheader("📊 Moving Averages (SMA 50 & SMA 100)")
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=filtered_df["ds"], y=filtered_df["SMA_50"], mode='lines', name='SMA 50'))
fig3.add_trace(go.Scatter(x=filtered_df["ds"], y=filtered_df["SMA_100"], mode='lines', name='SMA 100'))
st.plotly_chart(fig3)

# MACD Analysis
st.subheader("📊 MACD & Signal Line")
fig4 = go.Figure()
fig4.add_trace(go.Scatter(x=filtered_df["ds"], y=filtered_df["MACD"], mode='lines', name='MACD'))
fig4.add_trace(go.Scatter(x=filtered_df["ds"], y=filtered_df["Signal_Line"], mode='lines', name='Signal Line'))
st.plotly_chart(fig4)

# Volatility & Risk Metrics
st.subheader("📉 Rolling Volatility & Sharpe Ratio")
col1, col2 = st.columns(2)
with col1:
    fig5 = px.line(filtered_df, x="ds", y="Rolling_Volatility", title="Rolling Volatility")
    st.plotly_chart(fig5)
with col2:
    fig6 = px.line(filtered_df, x="ds", y="Sharpe_Ratio", title="Sharpe Ratio")
    st.plotly_chart(fig6)

# Conclusion
st.markdown("### Key Insights")
st.write("- The predicted Bitcoin price shows a general trend along with confidence intervals.")
st.write("- RSI fluctuations indicate overbought/oversold conditions.")
st.write("- SMA 50 vs. SMA 100 shows short-term vs. long-term momentum.")
st.write("- MACD & Signal Line crossover helps identify trend reversals.")
st.write("- Rolling volatility & Sharpe ratio provide insights into risk and returns.")

Overwriting dashboard.py
